In [1]:
#A lot of code from: https://ai.google.dev/gemma/docs/core/huggingface_text_finetune_qlora
%run verifyHuggingFacePkgs.py
#check if bfloat or float
if torch.cuda.get_device_capability()[0] >= 8:
    dtype = torch.bfloat16
    print("nice")
else:
    dtype = torch.float16

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


--- Bitsandbytes GPU Verification Script ---

Step 1: Checking for CUDA-enabled GPU...
✅ Success: CUDA is available. Found GPU: Orin

Step 2: Loading a model with 8-bit quantization (`load_in_8bit=True`)...


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


✅ Success: Model loaded in 8-bit without errors.
   This indicates that bitsandbytes is correctly installed and communicating with the GPU.

Step 3: Verifying model properties...
   - Model is on device: cuda:0
   ✅ Model is correctly placed on the CUDA device.
   - Model memory footprint: 165.54 MB

Step 4: Performing a simple inference test (forward pass)...
✅ Success: Forward pass completed without errors.

--- Verification Complete ---
🎉 All checks passed! Your `bitsandbytes` installation appears to be working correctly with your GPU.
nice


In [2]:
import os
import sys
# Get the absolute path of the current script's directory
current_dir = os.path.dirname(os.path.abspath("gemini2.5.ipynb"))

# Get the absolute path of the parent directory (project_folder)
parent_dir = os.path.dirname(current_dir)

# Add the parent directory to the Python path
sys.path.append(parent_dir)

# Now you can import from GetXY.py
from GetXY import x_string, y, expressions_not_in_x, y_test

# ... rest of your code
print("Successfully imported variables!")

-2 - -2 - 1
2543
-1.0

Expressions not in x:
0 - 2 - -5
True
1457
3.0
15
-4.0
[-5.   1.   1.   0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
  0.5]
Successfully imported variables!


In [3]:
x_test = expressions_not_in_x[:100]
y_test_string = [str(i) for i in y_test]
y_test_string = y_test_string[:100]
print(y_test[0], y_test_string[0])

3.0 3.0


In [4]:
y_string = []
for entry in y: 
    y_string.append(str(entry))
print(y_string[0])

-1.0


In [5]:
import datasets
print("success")

success


In [6]:
def create_conversation(x_list, y_list):
    conversations = []
    for x, y in zip(x_list, y_list):
        conversations.append({
            "messages": [
                {"role": "user", "content": x},
                {"role": "assistant", "content": y}
            ]
        })
    return conversations

# System message for the assistant
#system_message = ""

# User prompt that combines the user query and the schema
#user_prompt = ""


dataset = create_conversation(x_string, y_string)
val_dataset = create_conversation(x_test, y_test_string)
print(dataset[0])
from datasets import Dataset
dataset = Dataset.from_list(dataset)
val_dataset = Dataset.from_list(val_dataset)
print(dataset["messages"][0])

{'messages': [{'role': 'user', 'content': '-2 - -2 - 1'}, {'role': 'assistant', 'content': '-1.0'}]}
[{'content': '-2 - -2 - 1', 'role': 'user'}, {'content': '-1.0', 'role': 'assistant'}]


In [7]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_id = "google/gemma-3-1b-it"

# Define model init arguments
model_kwargs = dict(
    attn_implementation="eager", #flash_attention_2 or eager
    dtype=dtype, # What torch dtype to use, defaults to auto
    device_map="auto", # Let torch decide how to load the model
)

# BitsAndBytesConfig: Enables 4-bit quantization to reduce model size/memory usage
model_kwargs["quantization_config"] = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=model_kwargs['dtype'],
    bnb_4bit_quant_storage=model_kwargs['dtype'],
)

model = AutoModelForCausalLM.from_pretrained(model_id, **model_kwargs)
tokenizer = AutoTokenizer.from_pretrained(model_id)



# if tokenizer.pad_token_id is None:
    print("new")
    tokenizer.pad_token_id = tokenizer.eos_token_id

#tokenizer.apply_chat_template(dataset, tokenize = False)
def format_chat_template(example):
    conversation = example['messages']
    formatted_text = tokenizer.apply_chat_template(
        conversation,          # <-- Pass the list of messages for ONE conversation
        tokenize=False,
        add_generation_prompt=False # False for training examples
    )
    return {'text': formatted_text}
formatted_dataset = dataset.map(format_chat_template)
formatted_val_dataset = val_dataset.map(format_chat_template)

In [8]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    print("new")
def formatting_func_for_trainer(example):
    # This takes one example (a dict with a 'messages' key)
    # and returns the formatted string.
    text = tokenizer.apply_chat_template(
        example['messages'],
        tokenize=False,
        add_generation_prompt=False
    )
    return text

In [9]:
def preprocess_logits_for_metrics(logits, labels):
    """
    Original code from https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.preprocess_logits_for_metrics
    """
    if isinstance(logits, tuple):
        # Depending on the model and config, logits may be a tuple of tensors
        # here we take only the first tensor
        logits = logits[0]
    return logits.argmax(dim=-1)

In [10]:
from peft import LoraConfig

peft_config = LoraConfig(
    r=32,
    lora_alpha=32,
    target_modules=[
        "q_proj",
        "k_proj", 
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj"
    ],
    lora_dropout=0.05,
    task_type="CAUSAL_LM"
)


In [11]:
from trl import SFTConfig

args = SFTConfig(
    output_dir="big_output",         # directory to save and repository id
    max_length=64,                         # max sequence length for model and packing of the dataset
    packing=False,                           # Groups multiple samples in the dataset into a single sequence
    #num_train_epochs=3,                     # number of training epochs
    per_device_train_batch_size=2,          # batch size per device during training
    per_device_eval_batch_size = 2,
    gradient_accumulation_steps=4,          # number of steps before performing a backward/update pass
    eval_accumulation_steps=4,
    gradient_checkpointing=True,            # use gradient checkpointing to save memory
    gradient_checkpointing_kwargs={"use_reentrant": False},
    optim="adamw_torch",              # use fused adamw optimizer
    logging_steps = 10,                       # log every 10 steps
    eval_steps = 10,
    save_strategy="steps",
    eval_strategy="steps",
    learning_rate=1e-4,                     # learning rate, based on QLoRA paper
    lr_scheduler_kwargs={"min_lr": 1e-6},
    fp16=True if dtype == torch.float16 else False,   # use float16 precision
    bf16=True if dtype == torch.bfloat16 else False,   # use bfloat16 precision
    max_grad_norm=0.3,                      # max gradient norm based on QLoRA paper
    warmup_ratio=0.03,                      # warmup ratio based on QLoRA paper
    lr_scheduler_type="cosine_with_min_lr",           # use constant learning rate scheduler
    push_to_hub=False,                       # push model to hub
    report_to="tensorboard",                # report metrics to tensorboard
    dataset_kwargs={
        "add_special_tokens": False, # We template with special tokens
        "append_concat_token": True, # Add EOS token as separator token between examples
    },
    do_eval = True,
    load_best_model_at_end=True, # Recommended: loads the best ckpt at the end of training
    metric_for_best_model="accuracy", # <-- Tell the trainer to watch accuracy
    greater_is_better=True,         # <-- Tell the trainer that higher accuracy is better
)
model.gradient_checkpointing_enable()

In [12]:
#callback generated with gemini 2.5
from transformers import TrainerCallback, TrainingArguments, TrainerState, TrainerControl

class LearningRateLoggerCallback(TrainerCallback):
    def on_log(self, args: TrainingArguments, state: TrainerState, control: TrainerControl, logs=None, **kwargs):
        """
        Log the learning rate at the end of each epoch.
        """
        if logs is not None and "learning_rate" in logs:
            print(f"Epoch: {state.epoch:.2f}, Learning Rate: {logs['learning_rate']}")

In [13]:
print(tokenizer.eos_token)

<eos>


In [14]:
# mostly generated by mistral
import numpy as np
from transformers import PreTrainedTokenizer

def compute_metrics(eval_preds, tokenizer):
    preds, labels = eval_preds
    # Replace -100 (the ignore index) in labels with the pad_token_id for decoding
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    preds = np.clip(preds, 0, tokenizer.vocab_size - 1)
    #print(preds[0], labels[0])

    # Decode predicted tokens to text
    # We set skip_special_tokens=False to keep the template's special tokens for parsing
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=False)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=False)

    # Debugging: Print the first decoded prediction and label to see what they look like
    #print("----------- DEBUG -----------\n")
    #print(f"Sample Decoded Pred:\n'{decoded_preds[0]}'\n")
    #print(f"Sample Decoded Label:\n'{decoded_labels[0]}'\n")
    #print("---------------------------\n")

    # This is the separator that Gemma uses to indicate the start of the model's response
    turn_separator = "<start_of_turn>model\n"
    
    correct_predictions = 0
    total_predictions = 0

    for pred, label in zip(decoded_preds, decoded_labels):
        total_predictions += 1
        # We only care about the text that comes *after* the model's turn separator
        if turn_separator in label:
                      
            # Extract the response part from the label
            label_response = label.split(turn_separator, 1)[1]
            clean_label = label_response.replace("<end_of_turn>", "").replace("<pad>", "").strip()
            #print(f"nice: {clean_label}")

            # Extract the response part from the prediction
            if turn_separator in pred:
                pred_response = pred.split(turn_separator, 1)[1]
                clean_pred = pred_response.replace("<end_of_turn>", "") \
                                          .replace("<pad>", "") \
                                          .replace("<start_of_turn>", "") \
                                          .replace("model", "") \
                                          .strip()
                #print(f"ok: {clean_pred} nice: {clean_label}")
            else:
                # If the model didn't generate the turn separator, it's an incorrect prediction
                clean_pred = ""
                label_pred = "1"
            
            if clean_pred == clean_label:
                correct_predictions += 1

    accuracy = correct_predictions / total_predictions if total_predictions > 0 else 0.0
    return {"accuracy": accuracy}

from functools import partial

compute_metrics = partial(compute_metrics, tokenizer=tokenizer)

In [15]:
from trl import SFTTrainer

# Create Trainer object
trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=dataset,
    eval_dataset = val_dataset,
    formatting_func=formatting_func_for_trainer,
    compute_metrics = compute_metrics,
    peft_config=peft_config,
    preprocess_logits_for_metrics=preprocess_logits_for_metrics,
    processing_class = tokenizer
    #callbacks = [LearningRateLoggerCallback()]
)

Applying formatting function to train dataset:   0%|          | 0/2543 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/2543 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/2543 [00:00<?, ? examples/s]

Applying formatting function to eval dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

In [16]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 1}.


Step,Training Loss,Validation Loss,Accuracy,Entropy,Num Tokens,Mean Token Accuracy
10,10.792400,7.810430,0.000000,1.938616,1760.000000,0.247543
20,4.840800,3.047105,0.000000,1.905746,3525.000000,0.569590
30,2.076200,1.669594,0.050000,0.972585,5296.000000,0.675564
40,1.334600,1.093443,0.390000,1.042102,7067.000000,0.714480
50,0.833300,0.720414,0.700000,0.963523,8836.000000,0.760720
60,0.574300,0.505493,0.620000,0.555268,10600.000000,0.783029
70,0.458900,0.471937,0.720000,0.448455,12376.000000,0.775204
80,0.429900,0.455427,0.740000,0.459122,14140.000000,0.775684
90,0.444000,0.451514,0.730000,0.458259,15913.000000,0.769402
100,0.441200,0.426976,0.880000,0.477294,17675.000000,0.784618


TrainOutput(global_step=954, training_loss=0.5774555561177636, metrics={'train_runtime': 7785.3599, 'train_samples_per_second': 0.98, 'train_steps_per_second': 0.123, 'total_flos': 747372382963200.0, 'train_loss': 0.5774555561177636, 'entropy': 0.3759424462914467, 'num_tokens': 168369.0, 'mean_token_accuracy': 0.7805586121976376, 'epoch': 3.0})

In [17]:
trainer.save_model()
hub_model_id = "gemma-finetuned-math"
trainer.push_to_hub(hub_model_id)

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...events.1758058328.CapyJetson.1126.0: 100%|##########| 90.2kB / 90.2kB            

  ...nformers/big_output/tokenizer.model: 100%|##########| 4.69MB / 4.69MB            

  ...anformers/big_output/tokenizer.json: 100%|##########| 33.4MB / 33.4MB            

  ...ig_output/adapter_model.safetensors:   1%|          |  558kB /  104MB            

  ...events.1758401376.CapyJetson.1315.0:   1%|1         | 1.35kB / 90.2kB            

  ...ormers/big_output/training_args.bin:   1%|1         |  93.0B / 6.29kB            

CommitInfo(commit_url='https://huggingface.co/AntonBOOM/big_output/commit/40e4e66cca7e0ce8255c0cb549e780c09f6c93b9', commit_message='gemma-finetuned-math', commit_description='', oid='40e4e66cca7e0ce8255c0cb549e780c09f6c93b9', pr_url=None, repo_url=RepoUrl('https://huggingface.co/AntonBOOM/big_output', endpoint='https://huggingface.co', repo_type='model', repo_id='AntonBOOM/big_output'), pr_revision=None, pr_num=None)